## Ant Path Detection

The purpose of this tool is to extract the path that a single ant takes in navigating its way across a surface. This program takes in a vido and generates a position file for use in MATLAB. 

For the path extraction to be successful, there are several important actions that must be taken. 
1. The video must first be cropped appropriately so that the ant's path does not intersect any object that moved in the video, including people. 
2. Next, once the extracted path image is generated, the user must erase any other dark content in the image. This may be somewhat difficult and require trial and error. The edited image will be uploaded, and then the points will be generated. 
3. If there are points away from the ant path, this means that photo must be edited again (to remove the area with the points) and reuploaded. 

The points are an approximation of the ant path. 

## Video Instructions

Here are the instructions for uploading the video. Note that to upload a file you look at the four vertical icons on the left, click the bottom one several times, and then, from of the three horizontal icons that appear, click the leftmost one with the arrow.

1. See where movement intersects the full ant path.
2. Cut out up until there is no interfering movement.
3. Cut out everything after the ant leaves the board.
4. Create a zip file with the video and upload it.

## Execution Instructions

Click each of the black circles with play buttons in them. There are some where you will have to wait for it to complete. 

In [ ]:
# install components not present in Colab by default
!apt install imagemagick
!apt install ffmpeg
!pip3 install scikit-image

In [5]:
# import Python packages 
import subprocess
import os
import glob
import math 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import mean_shift
import cv2

from google.colab import files

In [1]:
frame_rate = "25" #@param ["15", "10", "20", "25"]
frame_rate = int(frame_rate)

In [ ]:
# the cropped + zipped video file you uploaded
video_file = glob.glob('*.zip')

# get the video's name and extension
video_name = video_file[0].split('/')[-1].split('.')[0]
ext = video_file[0].split('/')[-1].split('.')[1]

# unzip the upload zip file
subprocess.run(['unzip', video_file[0]])

# produce 15 frame image data from the video
subprocess.run(['ffmpeg', '-i', video_name+'.'+ext, '-r', frame_rate, '-q:v', '2', '-f', 'image2', f'./{video_name}_%d.jpeg'])

# create img directory again
if os.path.isdir(f'new_vi') != True:
  subprocess.run(['mkdir','new_vi'])

# renumber the images produced
imgs = glob.glob(f'*.jpeg')
largest_img = max([len(img.split('.')[0].split('_')[1]) for img in imgs])
for img in imgs:
  img_num = img.split('.')[0].split('_')[1]
  zeros = largest_img - len(img_num)
  new_num = ''.join(['0']*zeros)+img_num
  new_img = img.split('.')[0].split('_')[0]+'_'+str(new_num)+'.jpeg'
  subprocess.run(['mv', img, f'./new_vi/{new_img}'])

# foresground setup
c1 = ['convert', '-density','300']
imgs = sorted(glob.glob('./new_vi/*.jpeg'))
if len(imgs) % 2 != 0: # make file count even
  imgs = imgs[:-1]

# create img directory again
if os.path.isdir('result_imgs') != True:
  subprocess.run(['mkdir','r_imgs']) 

# step through 2 files at a time, generate the difference
for index in range(0, len(imgs), 2):
  c2 = ['-compose','difference','-composite','-colorspace','Gray']
  name_1st = './r_imgs/'+'result'+imgs[index].split('/')[-1].split('.')[0].split('_')[1]+'.jpeg'
  name_2nd = './r_imgs/'+'result'+imgs[index+1].split('/')[-1].split('.')[0].split('_')[1]+'.jpeg'
  c2.append(name_1st)
  c2.append(name_2nd)
  try:
    command = c1 + [imgs[index], imgs[index+1]] + c2
  except IndexError:
    pass 
  subprocess.run(command)

imgs = list(filter(lambda elt: 'result' in elt, sorted(glob.glob('./r_imgs/*.jpeg'))))
command_final = ['convert', '-compose','lighten', imgs[0]]
remaining_imgs = []
file_name = video_name+'.jpeg'
for img in imgs[1:]:
  remaining_imgs.append(img)
  remaining_imgs.append('-composite')
remaining_imgs.append(file_name)
command = command_final + remaining_imgs
subprocess.run(command)


# remove lesser image differences
T, threshInv = cv2.threshold(file_name, 55, 255, cv2.THRESH_BINARY)
invIMG = file_name+'_inv.jpeg'
cv2.imwrite(invImg, threshInv)

# download file
files.download(invImg)

### Next Step

So, the generated file has just been downloaded. Now is it time to remove the other white regions which are not part of the ant path using your photo edit tool of choice. Do this and reupload the file with the SAME name. 

For this next part, you will choose a number of points to approximate the ant path. Once you run the code chunk, observe the outputted image and see if there are any points flowing in space. If there are points, go back and edit the photo to remove these, and then reupload the photo, again with the same name. 


In [ ]:
img = Image.open(invImg)

arr = np.array(img.convert('L')).T[:,::-1]
# get indices where there line is the pixel values is dark, ie <100
indices = np.argwhere(arr < 100)

for x in np.arange(40,1,-1):
    print('{:.2f}: '.format(x), end='')
    print(len(mean_shift(indices, bandwidth=x)[0]))

points, labels = mean_shift(indices, bandwidth=3.0)

plt.scatter(points[:,0], points[:,1])



In [21]:
# img = cv2.imread('c0148.jpg')
# # plt.imshow(img)

# T, threshInv = cv2.threshold(img, 55, 255, cv2.THRESH_BINARY)
# cv2.imwrite(f'thresh.jpeg', threshInv)

True

In [ ]:
# from IPython.display import HTML, Image
# from google.colab.output import eval_js
# from base64 import b64decode

# # uncomment if above is run
# file_name = 'c0128.jpeg'

# canvas_html = """
# <canvas width=%d height=%d></canvas>
# <background
# <button>Finish</button>
# <script>
# var canvas = document.querySelector('canvas')
# var ctx = canvas.getContext('2d')
# ctx.lineWidth = %d
# var button = document.querySelector('button')
# var mouse = {x: 0, y: 0}
# canvas.addEventListener('mousemove', function(e) {
#   mouse.x = e.pageX - this.offsetLeft
#   mouse.y = e.pageY - this.offsetTop
# })
# canvas.onmousedown = ()=>{
#   ctx.beginPath()
#   ctx.moveTo(mouse.x, mouse.y)
#   canvas.addEventListener('mousemove', onPaint)
# }
# canvas.onmouseup = ()=>{
#   canvas.removeEventListener('mousemove', onPaint)
# }
# var onPaint = ()=>{
#   ctx.lineTo(mouse.x, mouse.y)
#   ctx.stroke()
# }
# var data = new Promise(resolve=>{
#   button.onclick = ()=>{
#     resolve(canvas.toDataURL('image/png'))
#   }
# })
# </script>
# """

# def draw(filename=file_name, w=400, h=200, line_width=1):
#   display(HTML(canvas_html % (w, h, line_width)))
#   data = eval_js("data")
#   binary = b64decode(data.split(',')[1])
#   with open(filename, 'wb') as f:
#     f.write(binary)
#   return len(binary)

# draw(file_name)



In [ ]:
# """ 
# https://half-6.github.io/lf-freehand-cropper/
# A promising example. Not exactly freehand, but choosing points is close enough. 

# https://netplayer.gr/crop/
# Same thing, the code is more simple
# """

# from IPython.display import HTML, Image
# from google.colab.output import eval_js
# from base64 import b64decode

# # uncomment if above is run
# file_name = 'c0128.jpeg'

# canvas_html = """
# <canvas width=%d height=%d></canvas>
# <background
# <button>Finish</button>
# <script>
# var canvas = document.querySelector('canvas')
# var ctx = canvas.getContext('2d')

# imageObj = new Image()
# imageObj.onload = function() {
#     ctx.drawImage(imageObj, 0, 0)

# };
# imageObj.src = "https://raw.githubusercontent.com/netplayer/crop/master/img.png"

# ctx.lineWidth = %d
# var button = document.querySelector('button')
# var mouse = {x: 0, y: 0}
# canvas.addEventListener('mousemove', function(e) {
#   mouse.x = e.pageX - this.offsetLeft
#   mouse.y = e.pageY - this.offsetTop
# })
# canvas.onmousedown = ()=>{
#   ctx.beginPath()
#   ctx.moveTo(mouse.x, mouse.y)
#   canvas.addEventListener('mousemove', onPaint)
# }
# canvas.onmouseup = ()=>{
#   canvas.removeEventListener('mousemove', onPaint)
# }
# var onPaint = ()=>{
#   ctx.lineTo(mouse.x, mouse.y)
#   ctx.stroke()
# }
# var data = new Promise(resolve=>{
#   button.onclick = ()=>{
#     resolve(canvas.toDataURL('image/png'))
#   }
# })
# </script>
# """

# def draw(filename=file_name, w=400, h=200, line_width=1):
#   display(HTML(canvas_html % (w, h, line_width)))
#   data = eval_js("data")
#   binary = b64decode(data.split(',')[1])
#   with open(filename, 'wb') as f:
#     f.write(binary)
#   return len(binary)

# draw(file_name)

In [ ]:
# # Voronoi Stippling with Inversin
# !git clone https://github.com/tmartin2/Rougier-2017.git

# !pip3 uninstall scipy
# !pip3 install scipy==1.1.0

# from PIL import Image
# import PIL.ImageOps    

# image = Image.open('P70.jpeg')
# inverted_image = PIL.ImageOps.invert(image)
# inverted_image.save('P70_1.jpeg')

# !python3 Rougier-2017/code/stippler.py --n_iter 20 --n_point 1000 --display 'P70_1.jpeg'

In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# from google.colab.patches import cv2_imshow

# #image = cv2.imread('P70.jpeg')
# #T, threshInv = cv2.threshold(image, 55, 255, cv2.THRESH_BINARY)
# image = cv2.imread('P70_1.jpeg')
# T, threshInv = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
# cv2_imshow(threshInv)


# # pixel_vals = threshInv.reshape((-1,3)) 
# # pixel_vals = np.float32(pixel_vals)

# # criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85) #criteria
# # k = 3 # Choosing number of cluster
# # retval, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS) 

# # centers = np.uint8(centers) # convert data into 8-bit values 
# # segmented_data = centers[labels.flatten()] # Mapping labels to center points( RGB Value)
# # segmented_image = segmented_data.reshape((image.shape)) # reshape data into the original image dimensions
# # plt.imshow(segmented_image)



In [ ]:
import numpy as np
import cv2
from google.colab import files

img = cv2.imread('c0148.jpg')
T, threshInv = cv2.threshold(img, 55, 255, cv2.THRESH_BINARY)
plt.imshow(img)


# gray = cv2.cvtColor(threshInv, cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# n_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)

# print(n_labels)

# size_thresh = 1
# for i in range(1, n_labels):
#     if stats[i, cv2.CC_STAT_AREA] >= size_thresh:
#         #print(stats[i, cv2.CC_STAT_AREA])
#         x = stats[i, cv2.CC_STAT_LEFT]
#         y = stats[i, cv2.CC_STAT_TOP]
#         w = stats[i, cv2.CC_STAT_WIDTH]
#         h = stats[i, cv2.CC_STAT_HEIGHT]
#         cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), thickness=1)
# cv2.imwrite('P70a.jpeg', img)
# plt.imshow(img)

# specify region to remove 

In [ ]:
# import numpy as np
# import cv2
# from google.colab import files

# img = cv2.imread('AL23.jpeg')
# T, threshInv = cv2.threshold(img, 55, 255, cv2.THRESH_BINARY)

# gray = cv2.cvtColor(threshInv, cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# n_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)

# print(n_labels)

# size_thresh = 1
# for i in range(1, n_labels):
#     if stats[i, cv2.CC_STAT_AREA] >= size_thresh:
#         #print(stats[i, cv2.CC_STAT_AREA])
#         x = stats[i, cv2.CC_STAT_LEFT]
#         y = stats[i, cv2.CC_STAT_TOP]
#         w = stats[i, cv2.CC_STAT_WIDTH]
#         h = stats[i, cv2.CC_STAT_HEIGHT]
#         cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), thickness=1)
# cv2.imwrite('AL.jpeg', img)
# plt.imshow(img)

# # specify region to remove 